# **Importing Necessary Library**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import random
import pathlib
import os
import seaborn as sns
from sklearn import metrics
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

# **Checking GPU Avaibility**

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print('GPU is available')
else:
    print('No GPU detected')

num_gpus = len(physical_devices)

if num_gpus > 0:
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {tf.config.experimental.get_device_details(physical_devices[0])}")
else:
    print("No GPUs available")

device = tf.device('gpu:0' if len(physical_devices) > 0 else 'cpu:0')

# **Dataset Path**

In [ ]:
data_dir = 'D:\Brain MRI Dataset\Dataset'
os.listdir(data_dir)

In [5]:
# Set the path to the dataset
dataset_path = data_dir

# Initialize empty lists for storing the images and labels
images = []
labels = []

# Loop over the subfolders in the dataset
for subfolder in os.listdir(dataset_path):
    
    subfolder_path = os.path.join(dataset_path, subfolder)
    if not os.path.isdir(subfolder_path):
        continue
  
  # Loop over the images in the subfolder
    for image_filename in os.listdir(subfolder_path):
       # Load the image and store it in the images list
        image_path = os.path.join(subfolder_path, image_filename)
        images.append(image_path)
    
        # Store the label for the image in the labels list
        labels.append(subfolder)
 
 # Create a pandas DataFrame from the images and labels
df = pd.DataFrame({'image': images, 'label': labels})

In [ ]:
# plot the classes
ax = sns.countplot(x=df.label)

# Set labels and titles
ax.set_xlabel("Name of Class")
ax.set_ylabel("The Number Of Samples for each class")

# Rotate x-axis labels if needed
plt.xticks(rotation=90)

plt.savefig('D:\Brain MRI Dataset\Bar Plot.pdf', bbox_inches='tight')

# Display the plot
plt.show()

# **Data Preprocessing**

In [ ]:
tf.random.set_seed(42)

train_data = keras.utils.image_dataset_from_directory(data_dir, validation_split = 0.1, subset = 'training', seed = 1, shuffle = True, batch_size = 16, image_size=(256,256))

test_data = keras.utils.image_dataset_from_directory(data_dir, validation_split = 0.1, subset = 'validation', seed = 1, shuffle = True, batch_size = 16, image_size=(256,256))

In [ ]:
filenames = pathlib.Path(data_dir)
for label in train_data.class_names :
    images = list(filenames.glob(f'{label}/*'))
    print(f'{label} : {len(images)}')

In [ ]:
train_data.cardinality().numpy(),  test_data.cardinality().numpy()

In [10]:
train_set = train_data.take(265)
val_set = train_data.skip(265)

In [ ]:
train_set.cardinality().numpy(), val_set.cardinality().numpy()

In [ ]:
# print random images from the train set
plt.figure(figsize = (15, 15))
for images, labels in train_set.take(1):
    for i in range(10):
        index = random.randint(0, len(images))
        ax = plt.subplot(4, 5, i + 1)
        plt.imshow(images[index].numpy().astype("uint8"))
        plt.title(train_data.class_names[labels[index]], color= 'blue', fontsize= 12)
        plt.axis(True)

plt.savefig('D:\Brain MRI Dataset\SampleImage.pdf',bbox_inches='tight')

plt.show()

In [ ]:
for images_batch, labels_batch in train_set:
    print(images_batch.shape)
    print(labels_batch.shape)
    break

# **Custom CNN Model Implementation**

In [16]:
tf.random.set_seed(42)

model = keras.Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(32,(3,3), activation='relu', input_shape=(256,256,3)), # Hidden Layer 1
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(128,(3,3), activation='relu'), # Hidden Layer 2
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(64,(3,3), activation='relu'), # Hidden Layer 3
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Flatten(), # Output layer
    layers.Dropout(0.5),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(37, activation='softmax')
])

In [17]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(), optimizer = keras.optimizers.Adam(learning_rate=1e-4), metrics = 'accuracy')

In [ ]:
history_1 = model.fit(train_set, epochs=60, validation_data=val_set)

In [ ]:
model.summary()

# **Saving Model**

In [29]:
# Save the model
model.save('D:\Brain MRI Dataset\Model\cnnModel.h5')

# **Loading Model**

In [31]:
model = load_model('D:\Brain MRI Dataset\Model\cnnModel.h5')

# **Training Performance Visualization**

In [20]:
def plot_training_curves(history_df):
    plt.figure(figsize = (13, 4), dpi = 120)
    ax = plt.subplot(1, 2, 1)
    plt.plot(range(1, len(history_df) + 1), history_df['loss'], marker = '.', label = 'Training Loss')
    plt.plot(range(1, len(history_df) + 1), history_df['val_loss'], marker = '^', label = 'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Cross Entropy')
    plt.grid()
    plt.legend()
    ax = plt.subplot(1, 2, 2) 
    plt.plot(range(1, len(history_df) + 1), history_df['accuracy'], marker = '.', label = 'Training Accuracy')
    plt.plot(range(1, len(history_df) + 1), history_df['val_accuracy'], marker = '^', label = 'Validation Accurcay')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid()
    plt.legend()
    plt.savefig('D:\Brain MRI Dataset\LossAccuracyGraph.pdf',bbox_inches='tight')
    plt.show()

In [ ]:
plot_training_curves(pd.DataFrame(history_1.history))

# **Model Evaluation**

In [ ]:
X_test, y_test = None, None
for images, labels in test_data:
    if X_test == None or y_test == None:
        X_test = images
        y_test = labels
    else:
        X_test = tf.concat([X_test, images], axis = 0)
        y_test = tf.concat([y_test, labels], axis = 0)
        
X_test.shape, y_test.shape

In [24]:
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
metrics.accuracy_score(y_test, y_pred)

In [ ]:
test_score = model.evaluate(test_data, verbose= 1)

print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

# **Classification Report**

In [ ]:
target_names = ['Brain Atrophy',
 'Brain Infection',
 'Brain Infection with abscess',
 'Brain Tumor',
 'Brain tumor (Astrocytoma Ganglioglioma)',
 'Brain tumor (Dermoid cyst craniopharyngioma)',
 'Brain Tumor (Ependymoma)',
 'Brain Tumor (Hemangioblastoma  Pleomorphic xanthroastrocytoma  metastasis)',
 'Brain tumor - Recurrenceremnant of previous lesion',
 'Brain tumor operated with ventricular hemorrhage',
 'Cerebral abscess',
 'Cerebral Hemorrhage',
 'cerebral venous sinus thrombosis',
 'demyelinating lesions',
 'Encephalomalacia with gliotic change',
 'focal pachymeningitis',
 'Glioma',
 'Hemorrhagic collection',
 'Ischemic change  demyelinating plaque',
 'Left Retro-orbital Haemangioma',
 'Leukoencephalopathy with subcortical cysts',
 'Malformation (Chiari I)',
 'meningioma',
 'Microvascular ischemic change',
 'Mid triventricular hydrocephalus',
 'NMOSD  ADEM',
 'Normal',
 'Obstructive Hydrocephalus',
 'pituitary tumor',
 'Post-operative Status with Small Hemorrhage',
 'Postoperative encephalomalacia',
 'small meningioma',
 'Small Vessel Diease Demyelination',
 'Stroke (Demyelination)',
 'Stroke (Haemorrhage)',
 'Stroke(infarct)',
 'White Matter Disease']
print(classification_report(y_test , y_pred, target_names=target_names))

# **Confusion Matrix**

In [ ]:
plt.figure(figsize = (15,15), dpi = 100)
sns.heatmap(metrics.confusion_matrix(y_test, y_pred), annot = True, fmt='d', cmap = 'Greens')
plt.xlabel('Predictions')
plt.ylabel('True Labels')
plt.title('Conusion Matrix')
plt.savefig('D:\Brain MRI Dataset\ConusionMatrix.pdf',bbox_inches='tight')
plt.show()

# **Predicting Some Random Sample Images**

In [37]:
# plot random images from a given dataset, and compare predictions with ground truth
def plot_random_predictions(dataset, model):

    shuffled_data = dataset.shuffle(10)
    class_names = dataset.class_names

    for images, labels in shuffled_data.take(1):
        plt.figure(figsize = (10, 10), dpi = 120)
        y_pred_proba = model.predict(images)

    for i in range(9):
        index = random.randint(0, len(images))
        ax = plt.subplot(3,3, i + 1)

        img = images[index].numpy().astype("uint8")
        y_true = class_names[labels[index]]
        y_pred = class_names[np.argmax(y_pred_proba[index], axis = 0)]
      
        c = 'g' if y_pred == y_true else 'r'
      
        plt.imshow(img)
        plt.title(f'Predicted : {y_pred}\nTrue label : {y_true}', c = c)
        plt.axis(False)
        plt.savefig('D:\Brain MRI Dataset\SamplePredictionImage.pdf',bbox_inches='tight')

In [ ]:
plot_random_predictions(test_data, model)

# **Explainable AI**

# **LIME**

**LIME (Local Interpretable Model-agnostic Explanations)** is a technique used to explain the predictions of machine learning models locally, meaning for individual predictions rather than the model as a whole. Its purpose is to provide insights into why a model made a specific prediction for a particular instance. This is especially important for complex models like deep neural networks, where understanding the reasoning behind individual predictions can be challenging.

* **Perturbation of Input Data:** LIME generates slightly modified versions of the input data by adding noise.

* **Model Prediction on Perturbed Data:** The black-box model makes predictions on these modified inputs.

* **Fitting a Simple Interpretable Model:** LIME trains a simpler, local model to approximate the predictions of the complex model for a specific input.

* **Feature Importance Extraction:** The simple model’s coefficients reveal which features were most influential for the original prediction.

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries

# Function for Displaying Original and LIME Images

In [ ]:
def plot_comparison(img, temp, mask, y_pred, y_true, limeEXP, c):
    fig = plt.figure(figsize = (15, 15), dpi = 120)

    ax = fig.add_subplot(142)
    ax.imshow(img)
    ax.set_title("Image")

    ax = fig.add_subplot(143)
    ax.imshow(mask)
    ax.set_title("Mask")
    ax.set_xlabel(f'Predicted : {y_pred}\nTrue label : {y_true}\nLime Explaination Class : {limeEXP}', c = c)

    ax = fig.add_subplot(144)
    ax.imshow(mark_boundaries(temp, mask))
    ax.set_title("Image+Mask Combined")

# LIME Function for Misclassified Images

In [ ]:
def Lime(dataset):

    shuffled_data = dataset.shuffle(10)
    class_names = dataset.class_names

    for images, labels in shuffled_data.take(1):
        y_pred_proba = model.predict(images)

    for i in range(30):
        index = random.randint(0, len(images))

        img = images[index].numpy().astype("uint8")
        y_true = class_names[labels[index]]
        y_pred = class_names[np.argmax(y_pred_proba[index], axis = 0)]

        if y_true != y_pred:
            explainer = lime_image.LimeImageExplainer()

            explanation = explainer.explain_instance(img, model.predict, top_labels=37, hide_color=0, num_samples=1000)

            for j in range(1):
                label = explanation.top_labels[j] 

                limeEXP = class_names[label]

                temp, mask = explanation.get_image_and_mask(label, positive_only=False, hide_rest=False)

                c = 'g' if y_pred == y_true else 'r'

                plot_comparison(img, temp, mask, y_pred, y_true, limeEXP, c)

# Visualization of LIME for Misclassified Images

In [ ]:
Lime(test_data)

# LIME Function for Correctly Classified Images

In [ ]:
def Lime(dataset):

    shuffled_data = dataset.shuffle(10)
    class_names = dataset.class_names

    for images, labels in shuffled_data.take(1):
        y_pred_proba = model.predict(images)

    for i in range(20):
        index = random.randint(0, len(images))

        img = images[index].numpy().astype("uint8")
        y_true = class_names[labels[index]]
        y_pred = class_names[np.argmax(y_pred_proba[index], axis = 0)]

        if y_true == y_pred:
            explainer = lime_image.LimeImageExplainer()

            explanation = explainer.explain_instance(img, model.predict, top_labels=37, hide_color=0, num_samples=1000)

            for j in range(1):
                label = explanation.top_labels[j] 

                limeEXP = class_names[label]

                temp, mask = explanation.get_image_and_mask(label, positive_only=False, hide_rest=False)

                c = 'g' if y_pred == y_true else 'r'

                plot_comparison(img, temp, mask, y_pred, y_true, limeEXP, c)

# Visualization of LIME for Correctly Classified Images

In [ ]:
Lime(test_data)